In [ ]:
#Created by Shlomi Kiko
#Goal: This program sends out the ID and Description of the products to OpenAI, and asks it to return a ranking.
#LinkedIn: https://www.linkedin.com/in/shlomikiko/

In [1]:
import pandas as pd
import sqlalchemy as alch
import urllib as lib
import numpy as np

In [2]:
server = 'localhost'

sourceDatabase = 'SourceTest'
targetDatabase = 'TargetTest'

targetSchema = 'dbo'
targetTable = targetSchema + '.' + 'OpenAITest'

driver = 'com.microsoft.sqlserver.jdbc.SQLServerDriver'
quotedSource = lib.parse.quote_plus('DRIVER={SQL SERVER NATIVE CLIENT 11.0};SERVER='+server+';DATABASE='+sourceDatabase+';Trusted_Connection=yes')
engineSource = alch.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quotedSource), future=True)

quotedTarget = lib.parse.quote_plus('DRIVER={SQL SERVER NATIVE CLIENT 11.0};SERVER='+server+';DATABASE='+targetDatabase+';Trusted_Connection=yes')
engineTarget = alch.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quotedTarget), future=True)

In [3]:
df_offersSummary = pd.read_sql_query("""
    SELECT TOP (5) [OfferID]
      ,[offerDescription]
  FROM SourceTest.[dbo].[OffersSummary]
""", engineSource)

In [4]:
df_offersSummary.dtypes

OfferID              int64
offerDescription    object
dtype: object

In [5]:
#OpenAI
import openai
from openai import OpenAI

client = OpenAI(api_key='')

In [7]:
headers = ['OfferID', 'Result']
df_finalTest = pd.DataFrame(columns=headers)

print(type(df_finalTest))

<class 'pandas.core.frame.DataFrame'>


In [8]:
df_offersSummary['Result'] = np.nan

In [ ]:
df_offersSummary

In [10]:
headers = ['OfferID', 'Tag', 'Weight']
df_final = pd.DataFrame(columns = headers)

In [ ]:
dictTest = {}

userPrompt = 'Give me 6 summarizing weighted tags for the input text as a json with keys named "tag" and "weight". please provide the weights between 0 and 50'

listFinal = []

for i in df_offersSummary.iterrows():
    offerID = df_offersSummary['OfferID'].iloc[i]
    offerDesc = df_offersSummary['offerDescription'].iloc[i]
    
    messageSysContent = offerDesc
    completion = client.chat.completions.create(
      model="gpt-4-1106-preview",
      messages=[
        {"role": "system", "content": f"{messageSysContent}"},
        {"role": "user", "content": f"{userPrompt}"}
      ]
    )
    result = completion.choices[0].message.content
    
    dictTest[offerID] = result

    dictTest2 = dictTest[offerID][10:]
    dictTest2 = dictTest2[: -6]
    dictTest2 = dictTest2.replace(' {"tag": "', '')
    dictTest2 = dictTest2.replace('{ "tag": "', '')
    splitLines = dictTest2.split('}')
    
    for i in splitLines:
        line = i
        
        tagIdx = line.find('",')
        tag = line[:tagIdx]
        tag = tag.replace(',', '')
        tag = tag.replace('\n', '')
        tag = tag.strip()
        
        weightIdx = line.find('"weight": ')
        weight = line[weightIdx + 10:]
        
        print(offerID, tag, weight)
        newRow = str(offerID) + '|' + tag + '|' + weight
        listFinal.append(newRow)

In [ ]:
#print(df_final)
print(listFinal)

In [ ]:
headers = ['OfferID', 'Tag', 'Weight']
df_final = pd.DataFrame({}, columns=headers)

In [ ]:
for i in listFinal:
    words = i.split('|')
    newRow = {'OfferID': words[0], 'Tag': words[1], 'Weight': words[2]}
    newRow = pd.DataFrame([{'OfferID': words[0], 'Tag': words[1], 'Weight': words[2]}])
    df_final = pd.concat([df_final, newRow])

In [ ]:
df_final = df_final.loc[df_final['Tag'] != '']

In [ ]:
print(df_final)

In [ ]:
df_final.to_sql('OpenAITest', con = engineTarget, schema = targetSchema, index = False, if_exists = 'replace')